[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichmondAlake/memorizz/blob/main/examples/memagents_multi_agents.ipynb)

In [1]:
! pip install -qU memorizz yahooquery

In [2]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [3]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [4]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

In [5]:
set_env_securely("VOYAGE_API_KEY", "Enter your Voyage AI API key: ")

### Step 1: Initalize a Memory Provider

A Memory Provider is a core abstraction layer that manages the persistence, organization, and retrieval of all memory components within an agentic system. It serves as the central nervous system for memory management, providing standardized interfaces between AI agents and underlying storage technologies.


In [6]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a mongodb config with voyageai embeddings
mongodb_config = MongoDBConfig(
    uri=os.environ["MONGODB_URI"],
    db_name="testing_memorizz",
    embedding_provider="voyageai",
    embedding_config={
        "embedding_type": "contextualized",
        "model": "voyage-context-3",
        "output_dimension": 256,
    }
)

# Create a memory provider
memory_provider = MongoDBProvider(mongodb_config)

/Users/richmondalake/miniconda3/envs/agent_memory_course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector search index 'vector_index' already exists for collection personas.
Vector search index 'vector_index' already exists for collection toolbox.
Vector search index 'vector_index' already exists for collection short_term_memory.
Vector search index 'vector_index' already exists for collection long_term_memory.
Vector search index 'vector_index' already exists for collection conversation_memory.
Vector search index 'vector_index' already exists for collection workflow_memory.
Vector search index 'vector_index' already exists for collection agents.
Vector search index 'vector_index' already exists for collection shared_memory.
Vector search index 'vector_index' already exists for collection summaries.


In [7]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [8]:
latitude = 40.7128
longitude = -74.0060
weather = get_weather(latitude, longitude)
print(weather)


29.9


In [9]:
from functools import lru_cache
from yahooquery import Ticker
import time

@lru_cache(maxsize=128)
def _fetch_price(symbol: str) -> float:
    """
    Internal helper to fetch the latest market price via yahooquery.
    Caching helps avoid repeated hits for the same symbol.
    """
    ticker = Ticker(symbol)
    # This returns a dict keyed by symbol:
    info = ticker.price or {}
    # regularMarketPrice holds the current trading price
    price = info.get(symbol.upper(), {}).get("regularMarketPrice")
    if price is None:
        raise ValueError(f"No price data for '{symbol}'")
    return price

def get_stock_price(
    symbol: str,
    currency: str = "USD",
    retry: int = 3,
    backoff: float = 0.5
) -> str:
    """
    Get the current stock price for a given symbol using yahooquery,
    with simple retry/backoff to handle occasional rate-limits.

    Parameters
    ----------
    symbol : str
        Stock ticker, e.g. "AAPL"
    currency : str, optional
        Currency code (Currently informational only; yahooquery returns native)
    retry : int, optional
        Number of retries on failure (default: 3)
    backoff : float, optional
        Backoff factor in seconds between retries (default: 0.5s)

    Returns
    -------
    str
        e.g. "The current price of AAPL is 172.34 USD."
    """
    symbol = symbol.upper()
    last_err = None
    for attempt in range(1, retry + 1):
        try:
            price = _fetch_price(symbol)
            return f"The current price of {symbol} is {price:.2f} {currency.upper()}."
        except Exception as e:
            last_err = e
            # simple backoff
            time.sleep(backoff * attempt)
    # if we get here, all retries failed
    raise RuntimeError(f"Failed to fetch price for '{symbol}' after {retry} attempts: {last_err}")


In [10]:
print(get_stock_price("AAPL"))

The current price of AAPL is 231.88 USD.


In [11]:
from memorizz import Toolbox
# Create a Toolbox instance
weather_toolbox = Toolbox(memory_provider=memory_provider)
stock_toolbox = Toolbox(memory_provider=memory_provider)
# Register the functions with the Toolbox
# These tools are now stored in the `ToolBox` store within the storage provider
weather_toolbox.register_tool(get_weather)
stock_toolbox.register_tool(get_stock_price)

'689ca0ad1018716aa5a6250a'

In [12]:
from memorizz import MemAgent
from memorizz.enums import ApplicationMode

# Create specialized agents
weather_agent = MemAgent(
    memory_provider=memory_provider,
    instruction="You are a weather specialist focused on weather data gathering and analysis.",
    application_mode=ApplicationMode.DEFAULT, # Default application mode is Assistant
)

weather_agent.add_tool(toolbox=weather_toolbox, persist=True)

weather_agent.save()

MemAgent(agent_id=689ca0b21018716aa5a6250b, memory_provider=<memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x111fb7410>)

In [13]:
stock_agent = MemAgent(
    memory_provider=memory_provider,
    instruction="You are a stock specialist focused on stock data gathering and analysis."
)
stock_agent.add_tool(toolbox=stock_toolbox)
stock_agent.save()

MemAgent(agent_id=689ca0b21018716aa5a6250c, memory_provider=<memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x111fb7410>)

In [14]:
# Create root agent with delegates
coordinator = MemAgent(
    memory_provider=memory_provider,
    delegates=[weather_agent, stock_agent],
    instruction="You coordinate complex tasks by delegating to specialized agents."
)

coordinator.save()

MemAgent(agent_id=689ca0b21018716aa5a6250d, memory_provider=<memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x111fb7410>)

In [15]:
# Execute multi-agent workflow
result = coordinator.run("Give me the weather in San Francisco and then the stock price of Apple")

In [16]:
result

'Here is the consolidated response based on the sub-task results:\n\nThe current weather in San Francisco is approximately 15.3°C. If you need additional weather details (such as wind, humidity, or specific conditions), please let me know.\n\nThe current stock price of Apple (AAPL) is $231.88 USD.\n\nIf you need further information about San Francisco’s weather or more detailed analysis of Apple’s stock, feel free to ask!\n\n**No gaps or conflicting information were found in the sub-task results.** The objective has been fully met based on the details provided.'